# 矩阵相乘

In [12]:
import numpy as np
import cupy as cp
import cupyx
import cv2
import string
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import  animation

In [13]:
def load_kernel(src,kname):
    _ = cp.zeros((1,)) # 初始化cuda context
    kernel = cp.core.core.compile_with_cache(src).get_function(kname)
    return kernel
def load_kernel_file(fname,kname):
    sources = None
    with open(fname,encoding='utf-8') as f:
        sources = f.read()
    kernel = load_kernel(sources,kname)
    return kernel

In [14]:
matrix_multiply = load_kernel_file('../../datas/cuda/10-matrix-multiply.cu','matrix_multiply_1')
matrix_elementwise_multiply = load_kernel_file('../../datas/cuda/10-matrix-multiply.cu','matrix_elementwise_multiply')

In [15]:
block_size = 16
rows_a,cols_a = 480,640
rows_b,cols_b = 640,480
A = cp.array(np.reshape(np.random.randint(0,100,(rows_a,cols_a)),(rows_a*cols_a,))).astype(np.float32)
B = cp.array(np.reshape(np.random.randint(0,100,(rows_b,cols_b)),(rows_b*cols_b,))).astype(np.float32)
D = cp.array(np.random.randint(0,100,(rows_a,cols_a))).astype(np.float32)
C = cp.zeros((rows_a*cols_b,),dtype=cp.float32)
E = cp.zeros((rows_a,cols_a),dtype=cp.float32)
print(C.shape)

(230400,)


In [16]:
rows_c,cols_c = rows_a,cols_b
block = (32,32,1)
grid = (int((cols_c - 1) / block[0] + 1),int((rows_c-1) / block[1] + 1),1)
print(grid)

(15, 15, 1)


In [17]:
matrix_multiply(grid=grid,block=block,args=(A,B,C,rows_a,cols_a,rows_b,cols_b,rows_c,cols_c))

In [18]:
dst_c = C.get().reshape((rows_a,cols_b))

In [19]:
# dst_nc = cp.dot(A.reshape(3,2),B.reshape(2,4)).get()
dst_nc = np.dot(A.reshape(rows_a,cols_a).get(),B.reshape(rows_b,cols_b).get())
print(dst_nc.shape)

(480, 480)


In [20]:
print((dst_c - dst_nc == 0).all())

True


In [21]:
block = (32,32,1)
grid = (int(cols_a / block[0]),int(rows_a / block[1]),1)

In [22]:
AA = A.reshape(rows_a,cols_a)
matrix_elementwise_multiply(grid=grid,block=block,args=(AA,D,E,cols_a,rows_a))

In [23]:
EE = np.multiply(AA.get(),D.get())

In [24]:
print((EE - E.get() == 0).all())

True
